In [1]:
import trafilatura
import requests
import lxml
from elasticsearch import Elasticsearch, helpers


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
vietnamnet = {
    "life": ["https://vietnamnet.vn/vn/doi-song/gia-dinh/",
             "https://vietnamnet.vn/vn/doi-song/chuyen-la/",
             "https://vietnamnet.vn/vn/doi-song/gioi-tre/",
             "https://vietnamnet.vn/vn/doi-song/tam-su/",
             "https://vietnamnet.vn/vn/doi-song/am-thuc/",
             "https://vietnamnet.vn/vn/doi-song/meo-vat/"],
    "news": ["https://vietnamnet.vn/vn/thoi-su/chong-tham-nhung/",
             "https://vietnamnet.vn/vn/thoi-su/quoc-hoi-khoa-xv/",
             "https://vietnamnet.vn/vn/thoi-su/an-toan-giao-thong/"
             "https://vietnamnet.vn/vn/thoi-su/moi-truong/",
             "https://vietnamnet.vn/vn/thoi-su/clip-nong/",
             "https://vietnamnet.vn/vn/thoi-su/bhxh-bhyt/",
             "https://vietnamnet.vn/vn/thoi-su/quoc-phong/"],
    "world": ["https://vietnamnet.vn/vn/the-gioi/binh-luan-quoc-te/",
              "https://vietnamnet.vn/vn/the-gioi/chan-dung/",
              "https://vietnamnet.vn/vn/the-gioi/ho-so/",
              "https://vietnamnet.vn/vn/the-gioi/the-gioi-do-day/",
              "https://vietnamnet.vn/vn/the-gioi/viet-nam-va-the-gioi/",
              "https://vietnamnet.vn/vn/the-gioi/quan-su/"]
}


In [4]:
vietnamnetLinks = {
    "life": [],
    "news": [],
    "world": []
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
for key, value in vietnamnet.items():
    for url in value:
        response = requests.get(url, headers=headers)
        tree = lxml.html.fromstring(response.content)
        title_elem = tree.cssselect('.vnn-title a')
        for elem in title_elem:
            vietnamnetLinks[key].append("https://vietnamnet.vn"+elem.get('href'))

print(len(vietnamnetLinks['life'])+len(vietnamnetLinks['news'])+len(vietnamnetLinks['world']))

496


In [5]:
vietnamnetContent = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in vietnamnetLinks.items():
    for url in value:
        if(url.startswith('https://vietnamnet.vn/')):
            id = url.split('-')[-1].replace('.html', '')
            downloaded = trafilatura.fetch_url(url)
            tree = lxml.html.fromstring(str(downloaded))
            date_elem = len(tree.cssselect("div.breadcrumb-box__time p span")) and tree.cssselect("div.breadcrumb-box__time p span")[0].text or ""
            content = trafilatura.extract(downloaded)
            content = content.replace('\n', ' ')
            item = {"domain": "https://vietnamnet.vn/", "URL": url,
                    "content": content, "date": date_elem, "id": id}
            vietnamnetContent[key].append(item)

print(len(vietnamnetContent['life'])+len(vietnamnetContent['news'])+len(vietnamnetContent['world']))
print(vietnamnetContent['life'][0])

496
{'domain': 'https://vietnamnet.vn/', 'URL': 'https://vietnamnet.vn/8x-ha-tinh-hoc-lom-o-xuong-moc-gio-noi-tieng-voi-tac-pham-doc-dao-827945.html', 'content': 'Tuấn Anh cho biết, bản thân đến với nghệ thuật điêu khắc là xuất phát từ đam mê. “Em mồ côi bố khi vừa lên 7 tuổi. Một mình mẹ phải nuôi 6 anh chị em nên cuộc sống rất khó khăn. Học xong bậc phổ thông (năm 2006), em vào Nam học nghề mộc mỹ nghệ. Nhưng vì phải lo cuộc sống cho gia đình nên sau 2 tháng em đành phải nghỉ học để đi làm. Giấc mơ trở thành nghệ nhân đồ mộc chấm dứt từ đó", Tuấn Anh kể. Để tiếp tục niềm đam mê nghệ thuật điêu khắc, thời gian rảnh rỗi Tuấn Anh thường đến các xưởng mộc để học lỏm từ các thợ. Khi đã tích lũy được một ít vốn trong quá trình đi làm thuê, năm 2014, 8X quyết định về quê mở xưởng vừa làm, vừa tự học. Nói là xưởng nhưng đồ nghề của Tuấn Anh lúc đó chỉ là những bộ cưa truyền thống và mấy cái đục thông dụng. Sản phẩm ban đầu anh làm ra là những vật dụng đơn giản, những bộ bàn ghế chạm khắc từ 

In [6]:
client = Elasticsearch("http://localhost:9200")

client.info()


/home/hplong/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'HoangPhiLong-20182651',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'jF2zfdDdQqueQik2u6h9Ug',
 'version': {'number': '7.17.2',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': 'de7261de50d90919ae53b0eff9413fd7e5307301',
  'build_date': '2022-03-28T15:12:21.446567561Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [ ]:
def gendata():
    for key,value in vietnamnetContent.items():
         for item in value:
            temp_item = {k: item[k] for k in set(list(item.keys())) - set(['id'])}
            temp_item['category'] = key
            client.index(index="journal", id=item['id'], document=temp_item)

gendata()